In [1]:
import mediapipe as mp
import numpy as np
import cv2
import os.path

# recording data

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [3]:
data = []

In [4]:
cap = cv2.VideoCapture(1)
with mp_hands.Hands(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        # Flip the image horizontally for a later selfie-view display, and convert
        # the BGR image to RGB.
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        results = hands.process(image)

        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:
            hand_landmarks = results.multi_hand_landmarks[0]
            mp_drawing.draw_landmarks(
                image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # appending to data
            data.append(list(np.array(
                [[i.x, i.y, i.z] for i in results.multi_hand_landmarks[0].landmark]).flatten()))

        # displaying captured frames
        cv2.putText(image, str(len(data)), (10, 30), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 0), 2, cv2.LINE_4)

        cv2.imshow('MediaPipe Hands', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()


In [5]:
len(data)

235

# saving data

In [6]:
number = 5
csv_path = os.path.join('data', '5-right.csv')

In [7]:
n_variables = len(data[0])
n_variables

63

In [8]:
for item in data:
    item.insert(0, number)

In [9]:
csv_headings = ['number'] + [f'v{i}' for i in range(n_variables)]

In [10]:
csv = [csv_headings] + [row for row in data]
len(csv)

236

In [11]:
with open(csv_path, 'w') as f:
    for row in csv:
        print(
            ','.join(
                str(field) for field in row
            ),
            file=f
        )